In [674]:
from itertools import permutations
def read_and_process_file(filename='input'):
    file1 = open(f'{filename}_17', 'r')
    data = [{">":1, "<":-1}[c] for c in file1.readlines()[0].replace("\n","")]
    return data

In [675]:
stones_def = [
    [[0,0], [0,1], [0,2], [0,3]],
    [[2,1], [1,0], [1,1], [1,2], [0,1]],
    [[2,2],[1,2],[0,0],[0,1],[0,2]],
    [[3,0],[2,0],[1,0],[0,0]],
    [[1,0],[1,1],[0,0],[0,1]]
]

In [676]:
def get_stone(structure):
    res = [[0 for _ in range(4)] for _ in range(4)]
    for s in structure:
        res[s[0]][s[1]] = 1
    return res

In [677]:
def print_stone(stone):
    for l in stone:
        print(" ".join([str(i) for i in l]))

In [678]:
def check_drop(stone, x, y, state, lowest_point):
    for s in stone:
        new_x = x + s[1]
        new_y = y + s[0] - 1 - lowest_point
        if new_y <= 0:
            return False
        if new_y >= len(state):
            continue
        if state[new_y][new_x]!=0:
            return False
    return True

In [679]:
def check_move(stone, x, y, move, state, lowest_point):
    for s in stone:
        new_x = x + s[1] + move
        new_y = y - lowest_point + s[0]
        if (new_x<0)|(new_x>=WIDTH):
            return False
        if new_y >= len(state):
            continue
        if state[new_y][new_x]!=0:
            return False
    return True

In [680]:
def update_state(stone, x, y, state, lowest_point, val):
    for s in stone:
        new_x = x + s[1]
        new_y = y - lowest_point + s[0]
        while(new_y >= len(state)):
            state.append([0 for _ in range(WIDTH)])
        state[new_y][new_x] = val   

    for i in range(len(state)-1, 0, -1):
        if all([state[i][j]>0 for j in range(WIDTH)]):
            lowest_point += i
            state = state[i:]
            return state, lowest_point
        
    return state, lowest_point

In [755]:
movements = read_and_process_file('input')
stones = {i:get_stone(stones_def[i]) for i in range(len(stones_def))}

In [756]:
def calculate_tower_height(limit_rocks):
    WIDTH = 7

    floor = {k:0 for k in range(WIDTH)}
    highest_point = 0
    lowest_point = 0
    state = [[0 for _ in range(WIDTH)]]
    num_of_processed_rocks = 0
    num_of_movements = 0
    
    while(num_of_processed_rocks<limit_rocks):
        current_rock = stones_def[num_of_processed_rocks%5]
        x = 2
        y = highest_point + 4
        while(True):
            current_movement = movements[num_of_movements%len(movements)]
            num_of_movements += 1
            if check_move(current_rock, x, y, current_movement, state, lowest_point):
                x += current_movement
            if check_drop(current_rock, x, y, state, lowest_point):
                y -= 1
            else:
                state, lowest_point = update_state(current_rock, x, y, state, lowest_point, 1)
                highest_point = lowest_point + len(state)-1
                break
        num_of_processed_rocks += 1
        _shape = get_floor_shape(state)
    return highest_point, state

In [757]:
res, state = calculate_tower_height(limit_rocks = 2022)
print(f"Current tower height: {res}")

Current tower height: 3100


In [822]:
def get_floor_shape(state, current_height):
    res = {}
    for c in range(len(state[0])):
        for i in range(len(state)-1, 0, -1):
            if state[i][c]>0:
                res[c] = current_height - i   
                break
                
    return " ".join([str(v) for k,v in res.items()])


In [823]:
def update_state(stone, x, y, state, lowest_point, val):
    for s in stone:
        new_x = x + s[1]
        new_y = y - lowest_point + s[0]
        while(new_y >= len(state)):
            state.append([0 for _ in range(WIDTH)])
        state[new_y][new_x] = val   
        
    return state, lowest_point

In [839]:
def calculate_tower_height(limit_rocks):
    WIDTH = 7

    floor = {k:0 for k in range(WIDTH)}
    highest_point = 0
    lowest_point = 0
    state = [[0 for _ in range(WIDTH)]]
    num_of_processed_rocks = 0
    num_of_movements = 0
    cycle = {}
    cycle_done = False
    additional_height = 0
    
    while(num_of_processed_rocks<limit_rocks):
        current_rock = stones_def[num_of_processed_rocks%5]
        x = 2
        y = highest_point + 4
        while(True):
            current_movement = movements[num_of_movements%len(movements)]
            num_of_movements += 1
            if check_move(current_rock, x, y, current_movement, state, lowest_point):
                x += current_movement
            if check_drop(current_rock, x, y, state, lowest_point):
                y -= 1
            else:
                state, lowest_point = update_state(current_rock, x, y, state, lowest_point, 1)
                highest_point = lowest_point + len(state)-1
                break
        num_of_processed_rocks += 1
        if not cycle_done:
            _shape = get_floor_shape(state, highest_point)
            cycle_check = (_shape,num_of_processed_rocks%5,num_of_movements%len(movements))
            if cycle_check in cycle:
                _gain_rocks = num_of_processed_rocks - cycle[cycle_check][0]
                _gain_height = highest_point - cycle[cycle_check][1]
                number_of_cycles_left = (limit_rocks-num_of_processed_rocks)//_gain_rocks
                num_of_processed_rocks += _gain_rocks*number_of_cycles_left
                additional_height = number_of_cycles_left*_gain_height
                cycle_done = True
            else:
                cycle[cycle_check] = (num_of_processed_rocks, highest_point)
    
    return highest_point + additional_height

In [840]:
res = calculate_tower_height(limit_rocks = 1000000000000)
print(f"Current tower height: {res}")

Current tower height: 1540634005751


In [841]:
assert res == 1540634005751

In [ ]:
#cos nie do konca rozumiem floor - czemu dziala zawsze jak tylko choc jeden kamien jest w kazdej kolumnie?